Нужно каким-то образом загрузить данные. И попробовать нарисовать на них bouding box-ы

In [15]:
import json

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import v2
from torchvision.transforms import functional


import torchvision.utils as utils
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


from PIL import Image

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import os
import json
import natsort

bboxes = []
# load json
with open('team_classification_data\\bboxes.json') as bboxes_json:
    bboxes = json.load(bboxes_json)

# load images 
data_dir = 'team_classification_data\\frames\\'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.jpeg']
sorted_filenames = natsort.natsorted(filenames)

image_tensors = [torchvision.io.read_image(data_dir + name) for name in sorted_filenames]
batch = torch.stack(image_tensors)

for bb, fn, tens in zip(bboxes.items(), sorted_filenames, image_tensors):
    print(f'{bb[0]} - {fn}')

    team_list = []
    bboxes_list = []
    for player_id, bbox in bb[1].items():
        height = tens.shape[1]
        width = tens.shape[2]

        x = bbox['box'][0] * width
        y = bbox['box'][1] * height
        w = bbox['box'][2] * width
        h = bbox['box'][3] * height

        bboxes_list.append([x, y, w, h])
        team_list.append(int(player_id))

    bboxes_tensor = torch.tensor(bboxes_list)
    print(bboxes_tensor.shape)
    print(bboxes_tensor)
    bboxes_tensor = torchvision.ops.box_convert(bboxes_tensor[1], 'xywh', 'xyxy')

    if bb[0] > '2000': 
        img = utils.draw_bounding_boxes(tens, bboxes_tensor[1], width=2, colors='green')
        img = transforms.ToPILImage()(img)
        img.show()
        break



In [17]:
class TeamClassificationDataset(Dataset):
    def __init__(self, img_folder_path, bboxes_file_path, transform = None):
        super().__init__()
        self.img_folder_path = img_folder_path
        self.transform = transform

        self.bboxes = {}
        with open(bboxes_file_path) as bboxes_json:
            self.bboxes = json.load(bboxes_json)
            self.bboxes = list(self.bboxes.items())
            # print ('type(self.bboxes) = ', type(self.bboxes))
            # print(self.bboxes[0])

    def __len__(self):
        return len(self.bboxes)

    def load_image(self, imgage_path):
        image = Image.open(imgage_path).convert('RGB')
        return image

    def __getitem__(self, index):
        img_path = f'{self.img_folder_path}\\{self.bboxes[index][0]}.jpeg'
        print(f'Loading {img_path}...')
        img = self.load_image(img_path)
        height = img.height
        width = img.width

        if self.transform:
            transformed_img = self.transform(img)

        # process bboxes
        team_list = []
        bboxes_list = []
        for id, player in self.bboxes[index][1].items():
            box = player['box']
            team = player['team']

            x = box[0] * width
            y = box[1] * height
            w = box[2] * width
            h = box[3] * height

            bboxes_list.append([x, y, w, h])
            team_list.append(int(team))

        bboxes_tensor = torch.tensor(bboxes_list)
        print(bboxes_tensor.shape)
        print(bboxes_tensor)
        bboxes_tensor = torchvision.ops.box_convert(bboxes_tensor, 'xywh', 'xyxy')

        debug_img = functional.pil_to_tensor(img)
        debug_img = utils.draw_bounding_boxes(debug_img, bboxes_tensor, width=2, colors='green')
        debug_img = transforms.ToPILImage()(debug_img)
        debug_img.show()

        return {'image': transformed_img, 
                'bboxes_tensor': bboxes_tensor,
                'teams_list': team_list}

transform = v2.Compose([
                v2.PILToTensor(), # uint8 [0, 255]
                v2.ToDtype(dtype=torch.float32, scale=True), # float32, [0, 1]
                v2.Normalize((0.5,), (0.5,)), # (img - mean) / std [-1, 1]
           ])

train_dataset = TeamClassificationDataset('team_classification_data\\frames',
                                          'team_classification_data\\bboxes.json',
                                           transform)

train_dataloader = DataLoader(train_dataset)

i = 0
for data in train_dataloader:
    i = i + 1
    if (i > 1):
        break

    # img = data['image']
    # # bbox = data['bbox']
    # # print(img.shape, bbox[1])
    # height = img.shape[2]
    # width = img.shape[3]

    # for team_id, box in bbox[1].items():
    #     bbox_list.append(normalize_bbox(height, width, box))

    # bboxes_tensor = torch.tensor(bbox_list)
    # bboxes_tensor = torchvision.ops.box_convert(bboxes_tensor[1], 'xywh', 'xyxy')

    # img = utils.draw_bounding_boxes(img, bboxes_tensor[1], width=2, colors='green')
    # if bbox[0][0] > '1':







# img, bbox = next(iter(train_dataloader))
# print(type(bbox))


# img = torch.squeeze(img, 0)
# print(img.shape, type(img))

# plt.imshow(  img.permute(1, 2, 0),  )

# img = transforms.ToPILImage()(img)
# img.show()


100
Loading team_classification_data\frames\1310.jpeg...
torch.Size([10, 4])
tensor([[ 147.6286,  255.9980,   81.8221,  152.6387],
        [ 871.8487,  313.4508,  130.9641,  221.5445],
        [1126.1875,  244.6214,   84.1326,  198.1680],
        [ 621.0183,  276.4618,   91.1209,  208.3889],
        [ 668.0269,  356.5368,  133.9814,  241.2749],
        [ 686.1049,  247.9799,   86.3285,  182.2844],
        [ 808.6389,  218.1206,   73.3393,  174.9938],
        [ 511.2910,  182.4466,   72.2001,  144.9039],
        [ 336.4641,  231.0955,   78.1841,  144.8021],
        [ 544.8488,  206.2736,   69.2758,  122.7992]])
<class 'PIL.Image.Image'> <class 'torch.Tensor'>
